# Vinho Verde: Wine Quality Analysis

In [3]:
library(tidyverse)
library(digest)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

# Introduction

In [4]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wine_red <- read_csv2(url)

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 1599 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (5): volatile acidity, citric acid, chlorides, density, sulphates
dbl (2): total sulfur dioxide, quality

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
colnames(wine_red) <- make.names(colnames(wine_red))
wine_red <- wine_red |>
            mutate(volatile.acidity = as.double(volatile.acidity),
                  citric.acid = as.double(citric.acid),
                  chlorides = as.double(chlorides),
                  alcohol = as.double(alcohol),
                  sulphates = as.double(sulphates)) |>
            select(-fixed.acidity, -free.sulfur.dioxide, -total.sulfur.dioxide, -density)

wine_red

volatile.acidity,citric.acid,residual.sugar,chlorides,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.70,0.00,19,0.076,351,0.56,94,5
0.88,0.00,26,0.098,32,0.68,98,5
0.76,0.04,23,0.092,326,0.65,98,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
0.510,0.13,23,0.076,342,0.75,11,6
0.645,0.12,2,0.075,357,0.71,102,5
0.310,0.47,36,0.067,339,0.66,11,6


In [6]:
wine_split <- initial_split(wine_red, prop = 0.75, strata = quality)
wine_train <- training(wine_split)
wine_test <- testing(wine_split)

wine_train

volatile.acidity,citric.acid,residual.sugar,chlorides,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.70,0.00,19,0.076,351,0.56,94,5
0.88,0.00,26,0.098,32,0.68,98,5
0.76,0.04,23,0.092,326,0.65,98,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
0.37,0.43,23,0.063,317,0.81,112,7
0.36,0.30,18,0.074,324,0.70,114,8
0.56,0.17,17,0.065,344,0.68,1055,7


In [10]:
means <- map_df(wine_train, mean)
means

volatile.acidity,citric.acid,residual.sugar,chlorides,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.5261603,0.2737229,25.93155,0.08898164,296.8589,0.6583639,1.766277e+12,5.631886
